In [304]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [305]:
# Identify file and read file path
file_path = 'Activities.csv'

df = pd.read_csv(file_path)
#df = pd.read_excel(file_path)
# df.head(2)

,Activity Type,Date,Favorite,Title,Distance,Calories,Time,Avg HR,Max HR,Avg Run Cadence,...,Total Reps,Dive Time,Min Temp,Surface Interval,Decompression,Best Lap Time,Number of Laps,Max Temp,Moving Time,Elapsed Time
0,Running,2023-05-11 19:15:41,False,Running,4.17,275,00:19:21,167,182,168,...,0,0:00,0.0,0:00,No,00:55.59,5,0.0,00:19:17,00:22:05
1,Running,2023-05-11 18:17:28,False,Running,5.07,338,00:24:00,165,181,166,...,0,0:00,0.0,0:00,No,00:21.30,6,0.0,00:23:59,00:24:00


Select relevant columns

In [306]:
columns_to_keep = [
  'Date', 'Title', 'Distance', 'Calories', 'Time', 'Avg HR', 'Max HR', 'Avg Run Cadence', 'Max Run Cadence', 'Avg Pace', 'Best Pace', 'Avg Stride Length', 'Moving Time', 'Elapsed Time'
  ]

In [307]:
df2 = df[columns_to_keep]
# df2.head(2)

Move column position

In [308]:
time = df2.pop('Time')
df2.insert(2, 'Time', time, False)

move_time = df2.pop('Moving Time')
df2.insert(3, 'Moving Time', move_time, False)


elapsed_time = df2.pop('Elapsed Time')
df2.insert(4, 'Elapsed Time', elapsed_time, False)

# df2.head(2)

#### Rename columns
-change <code>Date</code> to <code>datetime</code>  
-change <code>Title</code> to <code>Activity</code>  
-change all to Capital case  
-replace all ' ' with '_'  

In [309]:
# rename date and title
df2.rename(columns={"Date": "Date_time", "Title": "Activity"}, inplace=True)

C:\Users\hong\AppData\Local\Temp\ipykernel_5724\1212398620.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.rename(columns={"Date": "Date_time", "Title": "Activity"}, inplace=True)


In [310]:
# capitalise and replace spaces with underscores
df2.columns = df2.columns.str.replace(' ', '_')
df2.columns = df2.columns.str.capitalize()

# df2.head(2)

Make a copy of progress to new df

In [311]:
df3 = df2.copy()

Data wrangling on data fields (manipulate data types)

In [312]:
# First, replace all '--' with np.nan
df3.replace('--', value=np.nan, inplace=True)

# For calories, remove ','
df3['Calories'].replace(',', value='',  regex=True, inplace=True)

In [313]:
# convert Date_time column to datetime obj
df3['Date_time'] = pd.to_datetime(df3['Date_time'])

# convert calories, heart rate, cadence TO Int32 (note capitalisation) 
cols_to_int32 = ['Calories', 'Avg_hr', 'Max_hr', 'Avg_run_cadence', 'Max_run_cadence']
df3[cols_to_int32] = df3[cols_to_int32].astype('Int32')

# convert Distance, stride length to Float64 (note capitalisation)
cols_to_float64 = ['Distance', 'Avg_stride_length']
df3[cols_to_float64] = df3[cols_to_float64].astype('Float64')

# convert time variables to Timedelta objs (perhaps keep this as string)
cols_to_timedelta = ['Time', 'Moving_time', 'Elapsed_time']
df3[cols_to_timedelta] = df3[cols_to_timedelta].astype('timedelta64[s]')

NOTE:  
-pd.NA stores null Int values  
-np.NaN stores null Float values

In [314]:
df3.dtypes

Date_time            datetime64[ns]
Activity                     object
Time                 timedelta64[s]
Moving_time          timedelta64[s]
Elapsed_time         timedelta64[s]
Distance                    Float64
Calories                      Int32
Avg_hr                        Int32
Max_hr                        Int32
Avg_run_cadence               Int32
Max_run_cadence               Int32
Avg_pace                     object
Best_pace                    object
Avg_stride_length           Float64
dtype: object

Make a copy of progress to new df

In [315]:
df4 = df3.copy()

#### Data cleaning
Keep rows if:  
-Activity is <code>'Running'</code> or <code>'Walking'</code>  
-<code>Time</code> equal or more than 10 mins  
-<code>Calories</code> is not null(pd.NA)  
-<code>Distance</code> more than 200m

In [316]:
# Select rows if Activity is 'Walking' or 'Running'
before_activity = len(df4)
df4 = df4[(df4['Activity'] == 'Walking') | (df4['Activity'] == 'Running')]

# Select rows if Time is equal or more than 10 minutes
before_time = len(df4)
df4 = df4[df4['Time'] >= timedelta(minutes=10)]

# Drop row if Calories isnull
before_calories = len(df4)
df4['Calories'].dropna(inplace=True)

# Select rows if Distance is equal or more than 0.1 km
before_dist = len(df4)
df4 = df4[df4['Distance'] >= 0.2]

print = False
if print:
  print(f'Remove on Activity condition:\nRows dropped = {before_activity-len(df4)}  \nRemaining rows = {len(df4)}\n')
  print(f'Remove on Time condition:\nRows dropped = {before_time-len(df4)}  \nRemaining rows = {len(df4)}\n')
  print(f'Remove on Calorie condition:\nRows dropped = {before_calories-len(df4)}  \nRemaining rows = {len(df4)}\n')
  print(f'Remove on Distance condition:\nRows dropped = {before_dist-len(df4)}  \nRemaining rows = {len(df4)}\n')


Final cleaning:  
-For 'Avg_stride_length', replace 0.0 values with np.nan  
-convert datetime, all 3 time variables to string

In [317]:
# replace Avg_stride_length of 0.0 to np.nan
df4['Avg_stride_length'] = df4['Avg_stride_length'].replace(0.0, np.nan)

# convert date time to string format
df4['Date_time'] = df4['Date_time'].dt.strftime('%d/%b/%Y %I:%M %p')

# Convert time vars to string
time_cols_to_str = ['Time', 'Moving_time', 'Elapsed_time']
df4[time_cols_to_str] = df4[time_cols_to_str].astype('string')

# remove '0 days ' for time vars
df4['Time'].replace('0 days ', '', regex=True, inplace=True)
df4['Moving_time'].replace('0 days ', '', regex=True, inplace=True)
df4['Elapsed_time'].replace('0 days ', '', regex=True, inplace=True)

In [318]:
df4.head(3)

,Date_time,Activity,Time,Moving_time,Elapsed_time,Distance,Calories,Avg_hr,Max_hr,Avg_run_cadence,Max_run_cadence,Avg_pace,Best_pace,Avg_stride_length
0,11/May/2023 07:15 PM,Running,00:19:21,00:19:17,00:22:05,4.17,275,167,182,168,228,4:38,3:23,1.29
1,11/May/2023 06:17 PM,Running,00:24:00,00:23:59,00:24:00,5.07,338,165,181,166,214,4:44,3:47,1.28
2,30/Apr/2023 07:42 PM,Running,00:35:05,00:33:10,00:35:05,5.73,480,166,185,160,176,6:07,4:25,1.01


Save to excel sheet

In [319]:
df4.to_excel('activity_cleaned.xlsx', na_rep='', index=False)  